## Load dataset

In [1]:
!pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.5 MB/s eta 0:00:00


In [2]:
import numpy as np 
import pandas as pd 
import nltk
import spacy
import re
import underthesea as ud 
import json
from tqdm import tqdm

In [3]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional, for better lemmatization support
nltk.download('averaged_perceptron_tagger')  # For POS tagging
nltk.download('maxent_ne_chunker')  # For NER chunking
nltk.download('words')  # For supporting NER

!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!



  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.noun  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.noun  
  inflating: /usr/share/nltk_data/corpora/wordnet/adj.exc  


In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("haitranquangofficial/vietnamese-online-news-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/vietnamese-online-news-dataset


In [5]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/vietnamese-online-news-dataset/news_dataset.json


In [6]:
df = pd.read_json('/kaggle/input/vietnamese-online-news-dataset/news_dataset.json')

In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184539 entries, 0 to 184538
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id             184539 non-null  int64         
 1   author         184539 non-null  object        
 2   content        184539 non-null  object        
 3   picture_count  184539 non-null  int64         
 4   processed      184539 non-null  int64         
 5   source         184534 non-null  object        
 6   title          184539 non-null  object        
 7   topic          184539 non-null  object        
 8   url            184539 non-null  object        
 9   crawled_at     184125 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 14.1+ MB
None


In [8]:
# Remove empty data
df = df.dropna()
df = df[~(df == '').any(axis=1)].reset_index().drop(columns=['index'])

In [9]:
print(df.info())
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113418 entries, 0 to 113417
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id             113418 non-null  int64         
 1   author         113418 non-null  object        
 2   content        113418 non-null  object        
 3   picture_count  113418 non-null  int64         
 4   processed      113418 non-null  int64         
 5   source         113418 non-null  object        
 6   title          113418 non-null  object        
 7   topic          113418 non-null  object        
 8   url            113418 non-null  object        
 9   crawled_at     113418 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 8.7+ MB
None
(113418, 10)


In [10]:
df.head(3)

,id,author,content,picture_count,processed,source,title,topic,url,crawled_at
0,218269,(Nguồn: Sina),"Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...",1,0,vtc.vn,"Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G",Sống kết nối,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...,2022-08-01 09:09:21.181469
1,218268,Hồ Sỹ Anh,Kết quả thi tốt nghiệp THPT năm 2022 cho thấy ...,3,0,thanhnien.vn,Địa phương nào đứng đầu cả nước tổng điểm 3 mô...,Giáo dục,https://thanhnien.vn/dia-phuong-nao-dung-dau-c...,2022-08-01 09:09:15.311901
2,218267,Ngọc Ánh,Thống đốc Kentucky Andy Beshear hôm 31/7 cho h...,1,0,vnexpress,Người chết trong mưa lũ 'nghìn năm có một' ở M...,Thế giới,https://vnexpress.net/nguoi-chet-trong-mua-lu-...,2022-08-01 09:09:02.211498


In [11]:
df['content'][1]

'Kết quả thi tốt nghiệp THPT năm 2022 cho thấy điểm trung bình các môn toán, ngoại ngữ giảm, riêng ngữ văn tăng so với năm 2021. Về tổng điểm của 3 môn học năm 2022 là 18,13, giảm so với năm 2021 (18,92 điểm). Có 60 địa phương giảm tổng điểm 3 môn (chiếm 95,24%), chỉ có 3 địa phương tăng (chiếm 4,76%): Hòa Bình tăng 0,93 điểm, Hải Phòng tăng 0,57 và Vĩnh Phúc tăng 0,09. Có 5 địa phương giảm nhiều là Sóc Trăng (giảm 1,36 điểm), Bến Tre (1,37), Kon Tum (1,51), Lai Châu (1,53) và An Giang (1,61). Bình Dương có tổng 3 môn ngữ văn, toán và ngoại ngữ là 20,48 điểm, dẫn đầu cả nước. Tiếp theo là Hải Phòng (20,39 điểm), Nam Định (20,22), Vĩnh Phúc (20,01), Ninh Bình (19,82), TP.HCM (19,80), Hà Nam (19,46), Bắc Ninh (19,39), Hà Nội (19,25) và Tiền Giang (19,06)... Toàn quốc có 4 địa phương tổng điểm 3 môn trên 20 điểm (chiếm 6,35%), 21 địa phương từ 18 đến dưới 20 điểm (33,33%), 37 địa phương từ 15 đến dưới 18 điểm (58,73%) và 1 địa phương dưới 15 điểm (1,56%). 10 địa phương có tổng điểm 3 môn 

## Text Preprocessing

In [12]:
cols = ['author', 'title', 'topic', 'content']
articles = df[cols].to_dict(orient='records')
articles[:1]

[{'author': '(Nguồn: Sina)',
  'title': 'Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G',
  'topic': 'Sống kết nối',
  'content': 'Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số, Truyền thông và Truyền thông đại chúng Nga - Oleg Ivanov cho biết trong một cuộc phỏng vấn rằng, đến năm 2025\xa0Viện Khoa học & Công nghệ Skolkovo và đài phát thanh trực thuộc Bộ Kỹ thuật số Nga, Viện Khoa học Chế tạo sẽ nhận được khoản phân bổ 30 tỷ rúp để thúc đẩy các dự án chung về thiết bị liên lạc di động thế hệ thứ 6 (6G). Dự kiến, thiết bị 6G có thể được chế tạo trước năm 2025. Điều đó có nghĩa là giới chức Nga không còn quan tâm đến việc phát triển công nghệ 5G nữa mà đặt cược vốn và nguồn nhân lực của mình vào công nghệ 6G. Cách tiếp cận này rất táo bạo bởi nhìn chung, công nghệ truyền thông hiện đại cần phải tích lũy công nghệ từ thế hệ này sang thế hệ khác\xa0và sự phát triển giữa các thế hệ thường rất khó khăn. Điều khó khăn hơn nữa là 6G hiện đang trong giai đoạn nghiên cứu ban đầu trên khắp thế giới

In [13]:
import re
from underthesea import word_tokenize

class VietnamesePreprocessor:
    def __init__(self, stopwords=None):
        # Khởi tạo stopwords mặc định nếu không truyền vào
        if stopwords is None:
            stopwords = {
                "bị", "bởi", "các", "cái", "cần", "chỉ", "cho", "chưa",
                "có", "cùng", "cũng", "đã", "đang", "đây", "để", "đều", "do", "đó", "được",
                "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", "một", "này",
                "nên", "nếu", "nhiều", "như", "nhưng", "những", "nơi", "phải", "qua", "ra",
                "rằng", "rất", "rồi", "sau", "sẽ", "thì", "trên", "trước", "từ", "và",
                "vẫn", "vào", "vậy", "vì", "việc", "với", "tất_cả"
            }
        self.stopwords = stopwords

    def clean_text(self, text):
        """Chuẩn hóa text: lowercase, bỏ URL, ký tự đặc biệt."""
        if not isinstance(text, str):
            raise ValueError("Input must be a string")
        text = text.lower()
        text = re.sub(r"http\S+|www\S+", " ", text)  # xoá URL
        text = re.sub(r"[^0-9a-zàáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệ"
                      r"ìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữự"
                      r"ỳýỷỹỵđ\s]", " ", text)  # xoá ký tự ngoài bảng chữ
        text = re.sub(r"\s+", " ", text).strip()
        return text

    def tokenize(self, text):
        """Tách từ với underthesea."""
        return word_tokenize(text, format="text").split(" ")

    def remove_stopwords(self, tokens):
        """Loại bỏ stopwords."""
        return [w for w in tokens if w not in self.stopwords]

    def preprocess(self, text):
        """Pipeline tổng hợp: clean → tokenize → remove stopwords."""
        text = self.clean_text(text)
        tokens = self.tokenize(text)
        tokens = self.remove_stopwords(tokens)
        return tokens


In [14]:
preprocessor = VietnamesePreprocessor()

# tqdm hỗ trợ apply bằng cách wrap với progress_apply
tqdm.pandas()

df["processed_content"] = df["content"].progress_apply(preprocessor.preprocess)

100%|██████████| 113418/113418 [2:01:17<00:00, 15.59it/s]


In [15]:
# Xuất ra file CSV (giữ nguyên encoding tiếng Việt)
df.to_csv("processed_content.csv", index=False, encoding="utf-8-sig")

# Nếu muốn xuất Excel
df.to_excel("processed_content.xlsx", index=False)

# Nếu muốn xuất JSON
df.to_json("processed_content.json", orient="records", force_ascii=False, indent=2)
